In [2]:
import json
import numpy as np
import nltk
from pprint import pprint
from nltk.corpus import wordnet
from string import punctuation
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
import gensim 
from gensim.models import Word2Vec, KeyedVectors
import os
import re
from prettytable import PrettyTable
import pandas as pd

nltk.download("wordnet")
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()


[nltk_data] Downloading package wordnet to /home/vatsal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/vatsal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
top100 = {}
top100['travel'] = ['#travel', '#wanderlust', '#nature', '#travelling', '#traveling', '#traveller', '#photography', '#traveler', '#trip', '#travels', '#vacation', '#love', '#travelers', '#adventure', '#tourist', '#landscape', '#travellers', '#explore', '#holiday', '#beautiful', '#tourism', '#hiking', '#beach', '#photo', '#sunset', '#photographer', '#mountains', '#globetrotter', '#summer', '#art', '#sky', '#treking', '#europe', '#view', '#architecture', '#sea', '#happy', '#fun', '#city', '#sun', '#lifestyle', '#amazing', '#wanderer', '#italy', '#follow', '#backpacking', '#life', '#visiting', '#fashion', '#autumn', '#explorer', '#ocean', '#outdoors', '#india', '#world', '#mountain', '#beauty', '#spain', '#backpacker', '#style', '#like', '#france', '#exploring', '#trekking', '#clouds', '#asia', '#me', '#friends', '#usa', '#canon', '#happiness', '#blogger', '#holidays', '#ig', '#sunrise', '#smile', '#germany', '#girl', '#island', '#wander', '#paradise', '#turkey', '#discover', '#italia', '#voyage', '#flowers', '#landscapes', '#sightseeing', '#outdoor', '#history', '#indonesia', '#cute', '#forest', '#paris', '#food', '#australia', '#bali', '#pic', '#beaches', '#inspiration']
top100['food'] = ['#foodie', '#food', '#delicious', '#yummy', '#foodies', '#foods', '#eat', '#breakfast', '#dinner', '#tasty', '#cooking', '#lunch', '#homemade', '#dessert', '#love', '#eating', '#healthy', '#sweet', '#restaurant', '#photography', '#hungry', '#chef', '#blogger', '#travel', '#chocolate', '#baking', '#follow', '#cake', '#vegan', '#fresh', '#chicken', '#like', '#cook', '#amazing', '#blog', '#happy', '#favorite', '#brunch', '#coffee', '#weekend', '#fit', '#vegetarian', '#pasta', '#beautiful', '#pastry', '#fitness', '#gourmet', '#morning', '#desserts', '#seafood', '#eater', '#lifestyle', '#sweets', '#cafe', '#recipes', '#icecream', '#pizza', '#italy', '#culinary', '#candy', '#cheese', '#photographer', '#photo', '#cakes', '#recipe', '#noodles', '#diet', '#eats', '#rice', '#friends', '#cuisine', '#porridge', '#salad', '#nutrition', '#india', '#indonesia', '#bread', '#mornings', '#drinks', '#art', '#life', '#fun', '#gastronomy', '#sunday', '#cookies', '#kitchen', '#gym', '#me', '#beef', '#sushi', '#cupcakes', '#bake', '#spicy', '#saturday', '#fish', '#catering', '#burger', '#snack', '#music', '#delhi']
top100['baby'] = ['#baby', '#kids', '#love', '#babies', '#cute', '#family', '#mom', '#fashion', '#newborn', '#children', '#happy', '#beautiful', '#photography', '#girl', '#babys', '#adorable', '#sweet', '#motherhood', '#pregnancy', '#handmade', '#funny', '#lovely', '#smile', '#mommy', '#life', '#daughter', '#style', '#follow', '#sweetheart', '#like', '#pregnant', '#beauty', '#precious', '#kid', '#child', '#fun', '#girls', '#mummy', '#tiny', '#twins', '#amazing', '#toddler', '#model', '#photo', '#mama', '#toys', '#boy', '#sleep', '#matching', '#autumn', '#enjoy', '#friends', '#pretty', '#art', '#parenting', '#live', '#mother', '#me', '#amor', '#photographer', '#princess', '#fairy', '#sale', '#happiness', '#boys', '#boutique', '#lifestyle', '#portrait', '#fall', '#son', '#pink', '#nature', '#childhood', '#cuteness', '#maternity', '#cool', '#travel', '#angel', '#best', '#halloween', '#play', '#weekend', '#october', '#dad', '#innocent', '#brand', '#makeup', '#home', '#parenthood', '#nice', '#little', '#sunday', '#flowers', '#mum', '#canon', '#infant', '#party', '#wedding', '#summer', '#hot']
top100['jewellery'] = ['#jewellery', '#jewelry', '#fashion', '#accessories', '#earrings', '#necklace', '#style', '#jewels', '#handmade', '#beautiful', '#gemstone', '#trendy', '#gold', '#jewel', '#bracelet', '#rings', '#love', '#gems', '#silver', '#bracelets', '#design', '#ring', '#gemstones', '#crystals', '#shopping', '#stylish', '#art', '#pendant', '#diamond', '#cute', '#luxury', '#necklaces', '#bling', '#india', '#watches', '#handcrafted', '#designer', '#bijoux', '#trending', '#earring', '#bangles', '#bride', '#wedding', '#beauty', '#stone', '#tourmaline', '#charm', '#traditional', '#hyderabad', '#wholesaler', '#diamonds', '#bridal', '#photography', '#accessory', '#cabochons', '#tourmalines', '#stones', '#gem', '#goldplated', '#semiprecious', '#mumbai', '#silversmith', '#pendents', '#k', '#indian', '#sapphire', '#chennai', '#jewellers', '#girls', '#sale', '#jeweller', '#delhi', '#follow', '#fashionable', '#gift', '#bangalore', '#women', '#london', '#unique', '#ethnic', '#rawalpindi', '#crystal', '#vintage', '#oxidised', '#emerald', '#dubai', '#tucson', '#peridot', '#beads', '#makeup', '#saree', '#kunzite', '#chic', '#firecracker', '#pearl', '#lahore', '#choker', '#islamabad', '#wholesale', '#combo']
# top100['selfie'] = ['#me', '#love', '#follow', '#smile', '#photography', '#style', '#beautiful', '#fashion', '#happy', '#cute', '#fun', '#girl', '#life', '#portrait', '#like', '#pretty', '#photo', '#handsome', '#travel', '#nature', '#eyes', '#face', '#hair', '#summer', '#art', '#friends', '#amazing', '#model', '#food', '#beauty', '#pic', '#daily', '#girls', '#lifestyle', '#look', '#makeup', '#swag', '#photographer', '#fitness', '#autumn', '#cool', '#family', '#sky', '#picture', '#likes', '#sun', '#mood', '#music', '#followers', '#outfit', '#boy', '#colorful', '#self', '#motivation', '#lovely', '#gay', '#pose', '#landscape', '#black', '#inspiration', '#happiness', '#halloween', '#artist', '#gym', '#liker', '#pink', '#portraits', '#blogger', '#fit', '#sunset', '#wanderlust', '#design', '#funny', '#italy', '#traveling', '#hot', '#awesome', '#holiday', '#following', '#photos', '#india', '#night', '#blue', '#weekend', '#view', '#october', '#party', '#trip', '#red', '#tattoo', '#man', '#looks', '#stylish', '#sea', '#nice', '#good', '#beard', '#beach', '#woman', '#aesthetic']
top100['pet'] = ['#pet', '#pets', '#dog', '#cute', '#animals', '#love', '#dogs', '#cat', '#puppy', '#animal', '#cats', '#kitten', '#nature', '#kitty', '#meow', '#adorable', '#puppies', '#kittens', '#doggy', '#doggo', '#pup', '#photography', '#happy', '#funny', '#hound', '#eyes', '#beautiful', '#bunny', '#poodle', '#chihuahua', '#follow', '#rabbit', '#fluffy', '#paws', '#family', '#photo', '#lovely', '#chat', '#furry', '#daily', '#pomeranian', '#labrador', '#fun', '#autumn', '#sweet', '#maltese', '#smile', '#life', '#art', '#bunnies', '#hamster', '#baby', '#bird', '#me', '#world', '#bulldog', '#like', '#sleeping', '#tuesday', '#summer', '#parrots', '#husky', '#amor', '#rabbits', '#tot', '#zoo', '#doggie', '#meme', '#pug', '#istanbul', '#wildlife', '#beauty', '#adopt', '#girl', '#meowed', '#birds', '#purr', '#rescue', '#cavy', '#japan', '#doggies', '#halloween', '#photographer', '#woof', '#playtime', '#nice', '#portrait', '#relax', '#parakeet', '#singapore', '#parrot', '#fashion', '#fall', '#chihuahuas', '#friends', '#travel', '#morning', '#wolf', '#sweden', '#monday']
top100['art'] = ['#art', '#artist', '#drawing', '#artwork', '#illustration', '#sketch', '#painting', '#draw', '#sketchbook', '#creative', '#design', '#love', '#photography', '#beautiful', '#pencil', '#ink', '#illustrator', '#drawings', '#portrait', '#arts', '#gallery', '#artistic', '#fashion', '#paint', '#doodle', '#nature', '#picture', '#watercolor', '#graphic', '#color', '#photo', '#sketching', '#music', '#style', '#pen', '#anime', '#cute', '#paintings', '#follow', '#paper', '#graphics', '#artists', '#like', '#travel', '#abstract', '#tattoo', '#cartoon', '#girl', '#masterpiece', '#sketches', '#beauty', '#inspiration', '#procreate', '#designer', '#comics', '#artworks', '#photographer', '#life', '#digital', '#illustrations', '#model', '#handmade', '#watercolour', '#draws', '#happy', '#painter', '#black', '#colorful', '#colors', '#comic', '#fun', '#landscape', '#exhibition', '#cool', '#colour', '#architecture', '#aesthetic', '#acrylic', '#smile', '#graffiti', '#arty', '#tattoos', '#amazing', '#lifestyle', '#sculpture', '#blue', '#rap', '#halloween', '#inked', '#funny', '#fantasy', '#character', '#me', '#summer', '#singer', '#studio', '#logo', '#collage', '#doodles', '#pictures']
top100['architecture'] = ['#architecture', '#design', '#art', '#photography', '#building', '#travel', '#architect', '#city', '#buildings', '#urban', '#beautiful', '#style', '#minimal', '#street', '#architectural', '#interior', '#skyscraper', '#perspective', '#abstract', '#lines', '#town', '#architectures', '#geometric', '#designer', '#composition', '#geometry', '#arts', '#cities', '#home', '#landscape', '#nature', '#photo', '#photographer', '#interiors', '#house', '#italy', '#love', '#cityscape', '#modern', '#decor', '#pattern', '#luxury', '#sky', '#construction', '#architects', '#inspiration', '#arch', '#traveling', '#minimalism', '#render', '#wanderlust', '#sketch', '#rendering', '#europe', '#lifestyle', '#history', '#autumn', '#view', '#decoration', '#light', '#trip', '#amazing', '#concrete', '#paris', '#sunset', '#facade', '#artist', '#modernism', '#contemporary', '#sun', '#london', '#travelling', '#furniture', '#ig', '#life', '#france', '#church', '#traveller', '#canon', '#pic', '#explore', '#picture', '#details', '#colors', '#traveler', '#vacation', '#tourism', '#italia', '#milano', '#clouds', '#engineering', '#fashion', '#germany', '#spain', '#villa', '#wood', '#3d', '#exterior', '#blue', '#artwork']
top100['nature'] = ['#nature', '#photography', '#landscape', '#travel', '#beautiful', '#love', '#autumn', '#photo', '#sky', '#photographer', '#mountains', '#sunset', '#outdoors', '#clouds', '#art', '#flowers', '#beauty', '#tree', '#wanderlust', '#follow', '#amazing', '#adventure', '#trees', '#forest', '#view', '#explore', '#green', '#summer', '#life', '#sun', '#hiking', '#colors', '#wildlife', '#mountain', '#travelling', '#fashion', '#like', '#canon', '#trip', '#sea', '#ig', '#happy', '#water', '#natural', '#fall', '#outdoor', '#beach', '#style', '#traveling', '#fun', '#blue', '#landscapes', '#sunrise', '#flower', '#animals', '#lake', '#cute', '#picture', '#macro', '#me', '#river', '#earth', '#natures', '#friends', '#leaves', '#followers', '#vacation', '#plants', '#weather', '#smile', '#garden', '#india', '#girl', '#pic', '#november', '#photos', '#world', '#wild', '#birds', '#model', '#traveller', '#peace', '#lifestyle', '#bird', '#walk', '#animal', '#holiday', '#discover', '#trekking', '#portrait', '#morning', '#nice', '#sunday', '#ocean', '#sunsets', '#traveler', '#fitness', '#rain', '#italy', '#winter']

for topic in top100.keys():
    top100[topic] = [r.replace('#', '') for r in top100[topic]]

In [4]:
top100_mapping = {}
for key in top100.keys():
    top100_stemmed = [ps.stem(word) for word in top100[key]]
    top100_mapping[key]={}
    for i in range(0,100):
        top100_mapping[key][top100_stemmed[i]]=top100[key][i]

In [22]:
model = KeyedVectors.load_word2vec_format("./glove.6B/glove.6B.300d.w2vformat.txt", binary=False)

In [48]:
len(model.wv.vocab)

/home/vatsal/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


400000

In [26]:
final_dic = dict()

In [27]:
for topic in ["architecture","art","baby","food","jewellery","travel","nature","pet"]:
    for r, d, f in os.walk("./jsons/"+topic):
        print(topic)
        df = pd.read_csv('./csv/train/final_train_'+topic+".csv")
        dic = dict()
        for file in f:
            path = r+"/"+file
            print(path)
            if '.json' in file:
                file_ptr = open(path, "r")
                tmp_dic = json.load(file_ptr)
                dic.update(tmp_dic)
        final_dic.update(dic)

architecture
./jsons/architecture/modernarchitecture.json
./jsons/architecture/architecture.json
./jsons/architecture/architecturelovers.json
./jsons/architecture/architecturephotography.json
./jsons/architecture/architecture_hunter.json
./jsons/architecture/architectureporn.json
art
./jsons/art/artist.json
./jsons/art/artistsoninstagram.json
./jsons/art/art.json
./jsons/art/artoftheday.json
./jsons/art/drawing.json
./jsons/art/artwork.json
baby
./jsons/baby/baby.json
./jsons/baby/babyfashion.json
./jsons/baby/babycute.json
./jsons/baby/kids.json
./jsons/baby/babylove.json
./jsons/baby/babiesworld.json
food
./jsons/food/foodphotography.json
./jsons/food/foodporn.json
./jsons/food/foodstyling.json
./jsons/food/foodstagram.json
./jsons/food/foodblog.json
./jsons/food/foodaddict.json
jewellery
./jsons/jewellery/jewelerydesign.json
./jsons/jewellery/imitationjewellery.json
./jsons/jewellery/fashionjewelry.json
./jsons/jewellery/jewellery.json
travel
./jsons/travel/Luxarytravel.json
./jsons

In [28]:
def entry(line): 
    w, c = line.split("\t", 2)
    return (w, int(c))

dict_path = "./dict.txt"
dictionary = dict(entry(line) for line in open(dict_path))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))
cleanup = re.compile(r'[^a-z0-9]')

def word_prob(word): 
    return dictionary.get(word, 0) / total

def segment(text): 
    text = re.sub(cleanup, ' ', text)
    probs, lasts = [1.0], [0]
    for i in range(1,len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while i > 0:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words

In [29]:
def strip_punc(s):
    new_str = ""
    for c in s:
        if c in punctuation:
            new_str += " "
        else:
            new_str += c
    return new_str

In [30]:
def extract_hashtag(text):
    hashtags = []
    others = ""
    texts = text.split(" ")
    for t in texts:
        if t=="":
            continue
        if t[0]=='#':
            hashtags.append(t)
        else:
            others = others + t + " "
    return hashtags,others

In [55]:
x=0
x1=0
c=0
fans=0
# df = pd.read_csv('./csv/test/final_test.csv')
df = pd.read_csv('./csv/test/test-pred.csv')

for index,row in df.iterrows():
    
#     post_id = row['post_id'].split('.jpeg')[0]
    post_id = row['filename'].split('.jpeg')[0]
    try:
        input_sentence = final_dic[post_id]['text_des']
    except:
        x+=1
        continue
    
    hashtags,others = extract_hashtag(input_sentence)
    if(len(hashtags)<=3):
        x1+=1
        continue
    others = others + hashtags[0]
    others = others + " " + hashtags[1]
    others = others + " " + hashtags[2]
    del hashtags[0]
    del hashtags[0]
    del hashtags[0]
    
    preds = []
    for key,value in top100_mapping[topic].items():
        if len(preds)>10:
            break
        preds.append(value)
    tt = ""
    for h in hashtags:
        tt = tt+h
    text = strip_punc(tt.lower())
    toks = word_tokenize(text)
    toks_ = []
    for tok in toks:
        tok = segment(tok)
        for t in tok:
            if wordnet.synsets(t):
                t = ps.stem(t)
                if t not in stop_words and len(t)>2:
                    toks_.append(t)
    
    toks_ = set(toks_)
    toks_ = list(toks_)
    
    matr = np.empty((len(preds), len(toks_)))
#     print(preds)
#     print(toks_)
    for i, pred in enumerate(preds):
        for j, orig in enumerate(toks_):
            try:
                v1 = model.wv[ps.stem(pred)]
                v2 = model.wv[ps.stem(orig)]
#                 print("AA")
                similarity = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
                
            except:
#                 print("XX")
                similarity = 0
            matr[i][j] = similarity
    
    accs = []
    tabl = PrettyTable()
    tabl.field_names = ['Predicted'] + toks_
    for ind, row in enumerate(matr):
        l = [preds[ind]] + [round(i,2) for i in row]
        tabl.add_row(l)
        try:
            accs.append(max(row))
        except:
            accs.append(0)

    acc_ans = 0
    for acc in accs:
        if round(acc,1) >= 0.4:
            acc_ans += 1

    acc_ans=float(acc_ans)/float(len(preds))
    print(acc_ans)
    fans=fans+acc_ans
    c=c+1
#     break

/home/vatsal/.local/lib/python3.6/site-packages/ipykernel_launcher.py:57: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/home/vatsal/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


0.36363636363636365
0.36363636363636365
0.45454545454545453
0.45454545454545453
0.36363636363636365
0.45454545454545453
0.5454545454545454
0.45454545454545453
0.45454545454545453
0.2727272727272727
0.5454545454545454
0.18181818181818182
0.45454545454545453
0.2727272727272727
0.09090909090909091
0.45454545454545453
0.18181818181818182
0.0
0.36363636363636365
0.09090909090909091
0.5454545454545454
0.2727272727272727
0.5454545454545454
0.5454545454545454
0.45454545454545453
0.6363636363636364
0.0
0.2727272727272727
0.6363636363636364
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.45454545454545453
0.5454545454545454
0.45454545454545453
0.5454545454545454
0.2727272727272727
0.5454545454545454
0.45454545454545453
0.5454545454545454
0.09090909090909091
0.18181818181818182
0.45454545454545453
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.45454545454545453
0.6363636363636364
0.09090909090909091
0.0
0.2727272727272727
0.36363636363636365
0.09090909090909091
0.4545454545

0.2727272727272727
0.36363636363636365
0.6363636363636364
0.5454545454545454
0.36363636363636365
0.5454545454545454
0.2727272727272727
0.18181818181818182
0.5454545454545454
0.5454545454545454
0.36363636363636365
0.5454545454545454
0.09090909090909091
0.5454545454545454
0.2727272727272727
0.5454545454545454
0.2727272727272727
0.45454545454545453
0.5454545454545454
0.5454545454545454
0.45454545454545453
0.45454545454545453
0.5454545454545454
0.45454545454545453
0.5454545454545454
0.36363636363636365
0.5454545454545454
0.45454545454545453
0.36363636363636365
0.5454545454545454
0.36363636363636365
0.36363636363636365
0.18181818181818182
0.2727272727272727
0.6363636363636364
0.5454545454545454
0.5454545454545454
0.45454545454545453
0.5454545454545454
0.45454545454545453
0.36363636363636365
0.5454545454545454
0.36363636363636365
0.7272727272727273
0.18181818181818182
0.18181818181818182
0.45454545454545453
0.5454545454545454
0.5454545454545454
0.45454545454545453
0.18181818181818182
0.09090

0.36363636363636365
0.5454545454545454
0.45454545454545453
0.45454545454545453
0.5454545454545454
0.5454545454545454
0.45454545454545453
0.2727272727272727
0.36363636363636365
0.36363636363636365
0.5454545454545454
0.45454545454545453
0.18181818181818182
0.2727272727272727
0.18181818181818182
0.2727272727272727
0.45454545454545453
0.45454545454545453
0.18181818181818182
0.45454545454545453
0.45454545454545453
0.45454545454545453
0.36363636363636365
0.18181818181818182
0.36363636363636365
0.36363636363636365
0.18181818181818182
0.18181818181818182
0.2727272727272727
0.36363636363636365
0.45454545454545453
0.45454545454545453
0.2727272727272727
0.5454545454545454
0.2727272727272727
0.45454545454545453
0.6363636363636364
0.45454545454545453
0.5454545454545454
0.5454545454545454
0.45454545454545453
0.45454545454545453
0.6363636363636364
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.36363636363636365
0.45454545454545453
0.6363636363636364
0.36363636363636365


0.5454545454545454
0.5454545454545454
0.5454545454545454
0.18181818181818182
0.7272727272727273
0.0
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.18181818181818182
0.45454545454545453
0.5454545454545454
0.45454545454545453
0.6363636363636364
0.5454545454545454
0.09090909090909091
0.5454545454545454
0.6363636363636364
0.45454545454545453
0.5454545454545454
0.6363636363636364
0.45454545454545453
0.5454545454545454
0.5454545454545454
0.6363636363636364
0.45454545454545453
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.45454545454545453
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.6363636363636364
0.5454545454545454
0.45454545454545453
0.6363636363636364
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.36363636363636365
0.45454545454545453
0.5454545454545454
0.45454545454545453
0.2727272727272727
0.09090909090909091
0.36363636363636365
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.5454545454545454
0.09090909090909091
0.545454545

0.5454545454545454
0.45454545454545453
0.6363636363636364
0.5454545454545454
0.45454545454545453
0.5454545454545454
0.45454545454545453
0.6363636363636364
0.36363636363636365
0.2727272727272727
0.6363636363636364
0.45454545454545453
0.45454545454545453
0.2727272727272727
0.36363636363636365
0.5454545454545454
0.2727272727272727
0.45454545454545453
0.45454545454545453
0.45454545454545453
0.36363636363636365
0.45454545454545453
0.5454545454545454
0.45454545454545453
0.45454545454545453
0.09090909090909091
0.6363636363636364
0.2727272727272727
0.2727272727272727
0.6363636363636364
0.2727272727272727
0.5454545454545454
0.45454545454545453
0.36363636363636365
0.5454545454545454
0.6363636363636364
0.45454545454545453
0.36363636363636365
0.5454545454545454
0.09090909090909091
0.45454545454545453
0.6363636363636364
0.5454545454545454
0.5454545454545454
0.6363636363636364
0.36363636363636365
0.5454545454545454
0.18181818181818182
0.36363636363636365
0.5454545454545454
0.45454545454545453
0.3636

0.36363636363636365
0.2727272727272727
0.5454545454545454
0.45454545454545453
0.36363636363636365
0.6363636363636364
0.2727272727272727
0.36363636363636365
0.6363636363636364
0.09090909090909091
0.5454545454545454
0.45454545454545453
0.09090909090909091
0.45454545454545453
0.36363636363636365
0.6363636363636364
0.18181818181818182
0.36363636363636365
0.09090909090909091
0.45454545454545453
0.36363636363636365
0.2727272727272727
0.6363636363636364
0.09090909090909091
0.2727272727272727
0.45454545454545453
0.7272727272727273
0.36363636363636365
0.45454545454545453
0.5454545454545454
0.5454545454545454
0.2727272727272727
0.45454545454545453
0.09090909090909091
0.36363636363636365
0.6363636363636364
0.45454545454545453
0.5454545454545454
0.7272727272727273
0.09090909090909091
0.45454545454545453
0.6363636363636364
0.18181818181818182
0.5454545454545454
0.36363636363636365
0.36363636363636365
0.45454545454545453
0.36363636363636365
0.6363636363636364
0.36363636363636365
0.5454545454545454
0

In [56]:
print((fans/c)*100)

41.829336307863656
